Now the we try to get the formal definition of Differential Privacy and with that look at Global Differential Privacy. With Global Differential Privacy our goal is to find out how much noise to add to the output of a query to noise up the entire query or database with one block of noise.

We have two different measures called epsilon and delta which compose the threshold for leakage. Basically when we talk about Global Differential Privacy we want to add the noise to the output of a function on the database. This adding the noise after the computation is done because we now need to add less noise and get greater accuracy. This is the case because the function computation reduces the sensitivity to some extent too. Thus, if we can wait to add the noise till after the function computation is done we can get more accurate results.

Now we go through the formal definition of Differential Privacy as proposed by Cynthia Dwork:
![alt text](dp_formula.png "Title")

_Image From: "The Algorithmic Foundations of Differential Privacy" - Cynthia Dwork and Aaron Roth - https://www.cis.upenn.edu/~aaroth/Papers/privacybook.pdf_

Here, epsilon and delta comprise what is called as the Privacy Budget. Basically the above inequality means that the probability of leakage of information by our differentially private mechanism/algorithm, M, is epsilon. And the probability that we're gonna have a privacy leakage above epsilon is delta.

### How Much Noise Should Be Added

Now we're leavinng the idea of Local Differential Privacy behind. But for Global Differential Privacy we need to know how much noise should be added to the output of our query. Also, there is the question of what type of noise should be added, it can be Laplacian or Gaussian. The amount of noise should also be appropriate to satisfy our privacy budget(epsilon, delta). It is also affected by the sensitivity of the query that we're gonna perform. So, to sum up the amount of noise to be added to the output of our query depends on the following parameters:
- The type of noise(Laplacian/Gaussian)
- Sensitivity of our query function
- Epsilon value
- Delta value

Now here we're gonna work with Laplacian noise. Secondly the delta value is zero for Laplacian noise. Then the amount of Laplacian noise to be increased or decreased is scaled by a parameter called beta(b), which is given by, 
        
        b = sensitiviy(Query)/epsilon

So, now when we set b to this value we're gonna have a privacy leakage of <= epsilon.

# Project : Create A Differentially Private Query

In [1]:
# First we set our privacy budget, epsilon to 0.5
epsilon = 0.5

In [3]:
import numpy as np

# Time to bring in the required functions from previous projects

import torch

def get_parallel_db(db,remove_index):
    return torch.cat((db[0:remove_index], db[remove_index+1:]))

def get_parallel_dbs(db):
    parallel_dbs = list()
    
    for i in range(len(db)):
        parallel_dbs.append(get_parallel_db(db, i))
    
    return parallel_dbs

def create_db_and_pdbs(num_entries):
    
    db = (torch.rand(num_entries) > 0.5).float()
    pdbs = get_parallel_dbs(db)
    
    return db, pdbs

In [4]:
db, pdbs = create_db_and_pdbs(100)

In [8]:
# Now since we're working with databases filled with binary values we know that we get a sensitivity of 1 for the sum query
# Here's our sum query
def sum_query(db):
    return db.sum()

In [9]:
# Now we're ready to define our Laplacian Mecchanism
def laplacian_mechanism(db, query, sensitivity):
    
    beta = sensitivity/epsilon
    noise = torch.tensor(np.random.laplace(0, beta, 1))
    return query(db) + noise

In [10]:
# This is the true result of our sum query
sum_query(db)

tensor(49.)

In [14]:
# Now the result of the differentially private sum query which will be pretty close to our original query
laplacian_mechanism(db, sum_query, 1)

tensor([47.0640], dtype=torch.float64)

In [15]:
# Lets do it for the mean query
def mean_query(db):
    return torch.mean(db.float())

In [16]:
#Original mean
mean_query(db)

tensor(0.4900)

In [18]:
# Differentially Private mean query, the sensitivity will be 1/100 as the number of entries is 100
laplacian_mechanism(db, mean_query, 1/100)

tensor([0.4287], dtype=torch.float64)

Now if we decrease the epsilon to a very small value we will get very less privacy leakage. But this means that th output of our query, now, is all over the place, that is, we are adding a huge amount of noise to the output. What happens is that we get a very huge beta and the Laplacian distribution gets very wide.

In [19]:
epsilon = 0.0001

laplacian_mechanism(db, sum_query, 1)

tensor([20480.5128], dtype=torch.float64)

In [20]:
#Similarly for the mean query
laplacian_mechanism(db, mean_query, 1/100)

tensor([320.8145], dtype=torch.float64)